# 📌 Passo a Passo: Criando o Catálogo no Metastore do Databricks - Silver

In [0]:
%run "/Users/caio.santos.cavalheiro@gmail.com/00-Configuracao"

## 🔷 Criação Database




In [0]:
%sql
CREATE DATABASE IF NOT EXISTS catalogo_de_dados
LOCATION 's3://mvp-brasileirao-2024/silver/catalogo_de_dados';

Leitura das credenciais de forma segura pelo Spark 🔐


Caminhos definidos:
📂 Bronze: s3a://mvp-brasileirao-2024/bronze/ | 📂 Silver: s3a://mvp-brasileirao-2024/silver/ | 📂 Gold: s3a://mvp-brasileirao-2024/gold/


✅ Credenciais configuradas com sucesso!


✅ Conexão bem-sucedida! Arquivos encontrados na camada Bronze e Silver:


### 🔹 Registro das Tabela Delta no Metastore


In [0]:
%sql
DROP TABLE IF EXISTS default.brasil_seriea_2024_silver_todas_partidas;
CREATE TABLE default.brasil_seriea_2024_silver_todas_partidas
USING DELTA
LOCATION 's3://mvp-brasileirao-2024/silver/delta/brasil_seriea_2024_todas_partidas';

DROP TABLE IF EXISTS default.brasil_seriea_2024_silver_classificacao;
CREATE TABLE default.brasil_seriea_2024_silver_classificacao
USING DELTA
LOCATION 's3://mvp-brasileirao-2024/silver/delta/brasil_seriea_2024_classificacao';

DROP TABLE IF EXISTS default.brasil_seriea_2024_silver_estatistica_jogador;
CREATE TABLE default.brasil_seriea_2024_silver_estatistica_jogador
USING DELTA
LOCATION 's3://mvp-brasileirao-2024/silver/delta/brasil_seriea_2024_estatistica_jogador';


### 🔹 Validar o Registro no Metastore

Agora, vamos verificar se a tabela foi registrada corretamente.



In [0]:
%sql
SHOW TABLES IN default;


database,tableName,isTemporary
default,brasil_seriea_2024_classificacao,false
default,brasil_seriea_2024_estatistica_jogador,false
default,brasil_seriea_2024_silver_classificacao,false
default,brasil_seriea_2024_silver_estatistica_jogador,false
default,brasil_seriea_2024_silver_todas_partidas,false
default,brasil_seriea_2024_todas_partidas,false


# 📌 Adicionando o outro Catálogo - Catalogo Estatistico


## 🔷 brasil_seriea_2024_silver_classificacao

####🔹 Criando o Metadata do dataframe

In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import col, count, countDistinct, min, max, sum, expr

# Nome da tabela que queremos analisar (mudar para cada dataset da Silver)
tabela_silver = "default.brasil_seriea_2024_silver_classificacao"

# Ler os dados da Silver diretamente do Metastore
df = spark.read.table(tabela_silver)

# Criar DataFrame com estatísticas (incluindo contagem de nulos correta e duplicatas)
metadata_df = df.agg(
    *[count(col(c)).alias(f"{c}_count") for c in df.columns],  # Contagem total de registros NÃO NULOS
    *[countDistinct(col(c)).alias(f"{c}_distinct") for c in df.columns],  # Contagem de valores distintos
    *[min(col(c)).alias(f"{c}_min") for c in df.columns],  # Valor mínimo
    *[max(col(c)).alias(f"{c}_max") for c in df.columns],  # Valor máximo
    *[sum(col(c).isNull().cast("int")).alias(f"{c}_nulls") for c in df.columns],  # Contagem correta de nulos
    *[(count(col(c)) - countDistinct(col(c))).alias(f"{c}_duplicates") for c in df.columns]  # Contagem de valores duplicados
)


#### 🔹 Criação do Dicionário de descrições

In [0]:
col_descriptions = {
    "Posicao": ("Posição do clube na tabela de classificação", "Renomeação da coluna para padronização.", "Bronze → Silver"),
    "Clube": ("Nome do clube participante do campeonato", "Renomeação da coluna para padronização | Padronização dos nomes dos clubes para garantir consistência nos datasets.", "Bronze → Silver"),
    "Pontuacao": ("Pontos acumulados pelo clube na competição", "Renomeação da coluna para padronização.", "Bronze → Silver"),
    "Partidas_Jogadas": ("Número de partidas jogadas pelo clube", "Renomeação da coluna para padronização.", "Bronze → Silver"),
    "Vitorias": ("Quantidade de vitórias conquistadas pelo clube", "Renomeação da coluna para padronização.", "Bronze → Silver"),
    "Empates": ("Quantidade de empates do clube", "Renomeação da coluna para padronização.", "Bronze → Silver"),
    "Derrotas": ("Quantidade de derrotas do clube", "Renomeação da coluna para padronização.", "Bronze → Silver"),
    "Gols_Marcados": ("Gols marcados pelo clube ao longo da competição", "Renomeação da coluna para padronização.", "Bronze → Silver"),
    "Gols_Sofridos": ("Gols sofridos pelo clube ao longo da competição", "Renomeação da coluna para padronização.", "Bronze → Silver"),
    "Saldo_de_Gols": ("Saldo de gols do clube (Gols Marcados - Gols Sofridos)", "Renomeação da coluna para padronização.", "Bronze → Silver")
}


#### 🔹 Criando o Dataframe do Catálogo

In [0]:
# Criar lista de metadados para o Catálogo de Dados
catalog_data = []

# Iterar sobre as colunas da tabela Silver
for index, col_name in enumerate(df.columns, start=1):
    col_type = df.select(col(col_name)).schema.fields[0].dataType.simpleString()  # Tipo da coluna
    col_count = metadata_df.collect()[0][f"{col_name}_count"]  # Contagem total (não nulos)
    col_distinct = metadata_df.collect()[0][f"{col_name}_distinct"]  # Valores distintos
    col_nulls = metadata_df.collect()[0][f"{col_name}_nulls"]  # Contagem correta de valores nulos
    col_min = metadata_df.collect()[0][f"{col_name}_min"] if f"{col_name}_min" in metadata_df.columns else "N/A"
    col_max = metadata_df.collect()[0][f"{col_name}_max"] if f"{col_name}_max" in metadata_df.columns else "N/A"
    col_duplicates = metadata_df.collect()[0][f"{col_name}_duplicates"] if f"{col_name}_duplicates" in metadata_df.columns else 0  # Contagem de duplicados

    # Buscar definição, regra de transformação e linhagem dos dados no dicionário
    col_definition, transformations_applied, lineage_info = col_descriptions.get(
        col_name, (f"Descrição da coluna {col_name}", "Nenhuma alteração aplicada.", "Origem desconhecida")
    )

    # Definir observações sobre a coluna (pode ser preenchido manualmente se necessário)
    comments = "Sem observações adicionais."

    # Adicionar à lista
    catalog_data.append(Row(ID_Coluna=index,
                            Coluna=col_name, 
                            Tipo_de_Dado=col_type, 
                            Definição=col_definition,
                            Valores_Min_Max=f"{col_min} | {col_max}", 
                            Qtd_Valores_Registrados=col_count,
                            Qtd_Valores_Nulos=col_nulls,
                            Qtd_Valores_Distintos=col_distinct,
                            Qtd_Valores_Duplicados=col_duplicates,
                            Regras_de_Transformação=transformations_applied,
                            Linhagem_Dados=lineage_info,  # ✅ Agora vem direto do dicionário!
                            Observacoes=comments,
                            Fonte="Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil",
                            Link_Fonte="https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/",
                            Arquivo_Original="classificacao_brasileirao_2024.csv",
                            Arquivo_S3="BrasilSerieA_2024_ClassificacaoFinal.csv"
                            ))                            

# Criar o DataFrame do Catálogo de Dados
catalog_df = spark.createDataFrame(catalog_data)


#### 🔹 Salvando o Catálogo no S3 camada _Bronze_

In [0]:
# Caminho no S3 para salvar o Catálogo Estatístico
delta_catalogo_estatistico_path = "s3://mvp-brasileirao-2024/silver/delta/catalog_classificacao"

dbutils.fs.rm(delta_catalogo_estatistico_path, recurse=True)
print("🚨 Arquivos antigos do Delta removidos com sucesso!")

# Salvar a tabela Delta com as estatísticas
catalog_df.write.format("delta").mode("overwrite").save(delta_catalogo_estatistico_path)

print("✅ Catálogo Estatístico salvo no formato Delta com sucesso!")

🚨 Arquivos antigos do Delta removidos com sucesso!
✅ Catálogo Estatístico salvo no formato Delta com sucesso!


#### 🔹 Criando Tabela de Catálogos no Hive

In [0]:
%sql
DROP TABLE IF EXISTS catalogo_de_dados.catalog_classificacao_silver;

CREATE TABLE catalogo_de_dados.catalog_classificacao_silver
USING DELTA
LOCATION 's3://mvp-brasileirao-2024/silver/delta/catalog_classificacao';

#### 🔹 Exibindo o Catálogo de Dados via SQL

In [0]:
%sql
select * from catalogo_de_dados.catalog_classificacao_silver

ID_Coluna,Coluna,Tipo_de_Dado,Definição,Valores_Min_Max,Qtd_Valores_Registrados,Qtd_Valores_Nulos,Qtd_Valores_Distintos,Qtd_Valores_Duplicados,Regras_de_Transformação,Linhagem_Dados,Observacoes,Fonte,Link_Fonte,Arquivo_Original,Arquivo_S3
2,Clube,string,Nome do clube participante do campeonato,Athletico-PR | Vasco da Gama,20,0,20,0,Renomeação da coluna para padronização | Padronização dos nomes dos clubes para garantir consistência nos datasets.,Bronze → Silver,Sem observações adicionais.,Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil,https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/,classificacao_brasileirao_2024.csv,BrasilSerieA_2024_ClassificacaoFinal.csv
9,Gols_Sofridos,int,Gols sofridos pelo clube ao longo da competição,29 | 61,20,0,18,2,Renomeação da coluna para padronização.,Bronze → Silver,Sem observações adicionais.,Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil,https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/,classificacao_brasileirao_2024.csv,BrasilSerieA_2024_ClassificacaoFinal.csv
10,Saldo_de_Gols,int,Saldo de gols do clube (Gols Marcados - Gols Sofridos),-29 | 30,20,0,17,3,Renomeação da coluna para padronização.,Bronze → Silver,Sem observações adicionais.,Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil,https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/,classificacao_brasileirao_2024.csv,BrasilSerieA_2024_ClassificacaoFinal.csv
4,Partidas_Jogadas,int,Número de partidas jogadas pelo clube,38 | 38,20,0,1,19,Renomeação da coluna para padronização.,Bronze → Silver,Sem observações adicionais.,Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil,https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/,classificacao_brasileirao_2024.csv,BrasilSerieA_2024_ClassificacaoFinal.csv
5,Vitorias,int,Quantidade de vitórias conquistadas pelo clube,6 | 23,20,0,15,5,Renomeação da coluna para padronização.,Bronze → Silver,Sem observações adicionais.,Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil,https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/,classificacao_brasileirao_2024.csv,BrasilSerieA_2024_ClassificacaoFinal.csv
8,Gols_Marcados,int,Gols marcados pelo clube ao longo da competição,29 | 61,20,0,15,5,Renomeação da coluna para padronização.,Bronze → Silver,Sem observações adicionais.,Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil,https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/,classificacao_brasileirao_2024.csv,BrasilSerieA_2024_ClassificacaoFinal.csv
3,Pontuacao,int,Pontos acumulados pelo clube na competição,30 | 79,20,0,17,3,Renomeação da coluna para padronização.,Bronze → Silver,Sem observações adicionais.,Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil,https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/,classificacao_brasileirao_2024.csv,BrasilSerieA_2024_ClassificacaoFinal.csv
1,Posicao,int,Posição do clube na tabela de classificação,1 | 20,20,0,20,0,Renomeação da coluna para padronização.,Bronze → Silver,Sem observações adicionais.,Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil,https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/,classificacao_brasileirao_2024.csv,BrasilSerieA_2024_ClassificacaoFinal.csv
7,Derrotas,int,Quantidade de derrotas do clube,5 | 22,20,0,12,8,Renomeação da coluna para padronização.,Bronze → Silver,Sem observações adicionais.,Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasi

## 🔷 brasil_seriea_2024_silver_todas_partidas

####🔹 Criando o Metadata do dataframe

In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import col, count, countDistinct, min, max, sum

# Nome da tabela que queremos analisar (mudar para cada dataset da Silver)
tabela_silver = "default.brasil_seriea_2024_silver_todas_partidas"

# Ler os dados da Silver diretamente do Metastore
df = spark.read.table(tabela_silver)

# Criar DataFrame com estatísticas (incluindo contagem de nulos correta e duplicatas)
metadata_df = df.agg(
    *[count(col(c)).alias(f"{c}_count") for c in df.columns],  # Contagem total de registros NÃO NULOS
    *[countDistinct(col(c)).alias(f"{c}_distinct") for c in df.columns],  # Contagem de valores distintos
    *[min(col(c)).alias(f"{c}_min") for c in df.columns],  # Valor mínimo
    *[max(col(c)).alias(f"{c}_max") for c in df.columns],  # Valor máximo
    *[sum(col(c).isNull().cast("int")).alias(f"{c}_nulls") for c in df.columns],  # Contagem correta de nulos
    *[(count(col(c)) - countDistinct(col(c))).alias(f"{c}_duplicates") for c in df.columns]  # Contagem de valores duplicados
)

#### 🔹 Criação do Dicionário de descrições

In [0]:
col_descriptions = {
    # Colunas originais da Bronze (renomeadas na Silver)
    "Pais": ("País onde o campeonato foi disputado", "Renomeação da coluna para padronização", "Bronze → Silver"),
    "Liga": ("Nome da liga do campeonato", "Renomeação da coluna para padronização", "Bronze → Silver"),
    "Temporada": ("Ano da temporada do campeonato", "Renomeação da coluna para padronização | Filtragem para manter apenas 2024", "Bronze → Silver"),
    
    # 🔹 Atualização da coluna "Data"
    "Data": ("Data em que a partida foi disputada", 
             "Renomeação da coluna e conversão para formato de data | Ajuste das datas das partidas não encontradas na base Estatistica por Jogador (-1 dia, quando necessário)", 
             "Bronze → Silver"),
    
    "Horario": ("Horário da partida", "Renomeação da coluna para padronização", "Bronze → Silver"),
    
    # Novas colunas criadas na Silver
    "Match_ID": ("Identificador único da partida", "Criado concatenando 'Match_' com um ID incremental", "Silver (Coluna nova)"),
    "Trimestre": ("Trimestre em que a partida foi disputada", "Criado a partir da data do jogo", "Silver (Coluna nova)"),
    "Turno": ("Indicação se a partida foi no primeiro ou segundo turno do campeonato", "Criado com base na data do jogo", "Silver (Coluna nova)"),
    
    # Transformações para análise individual de cada time
    "Clube": ("Nome do time participante da partida (mandante ou visitante)", "Criado a partir das colunas Home e Away, e unificando em uma única coluna", "Silver (Transformação)"),
    "MandanteVisitante": ("Indica se o time jogou em casa ('Mandante') ou fora ('Visitante')", "Transformado a partir dos DataFrames 'df_home' e 'df_away' e criado a partir da união de ambos", "Silver (Transformação)"),
    "Goals": ("Número de gols marcados pelo time na partida", "Criado unificando HG (gols do mandante) e AG (gols do visitante)", "Silver (Coluna nova)"),
    
    # Transformações aplicadas à coluna Resultado
    "Resultado": ("Resultado da partida do ponto de vista do time (Vitória, Derrota ou Empate)", "Ajustado conforme a posição do time na partida", "Silver (Transformação)"),
    
    # 🔹 Nova coluna "Status_Ajuste"
    "Status_Ajustes": ("Indica se a data da partida foi ajustada", 
                      "Criado para identificar alterações na data das partidas. Valores possíveis: 'Ajustado -1 dia' ou 'Sem modificação'", 
                      "Silver (Coluna nova)")
}


In [0]:
observacoes = {
    "Data": "⚠️ Ajuste manual: Datas ajustadas para garantir a correspondência entre os arquivos de partidas e estatísticas. Quando necessário, foi subtraído 1 dia das partidas não encontradas na estatística."
}

#### 🔹 Criando o Dataframe do Catálogo

In [0]:
# Criar lista de metadados para o Catálogo de Dados
catalog_data = []

# Iterar sobre as colunas da tabela Silver
for index, col_name in enumerate(df.columns, start=1):
    col_type = df.select(col(col_name)).schema.fields[0].dataType.simpleString()  # Tipo da coluna
    col_count = metadata_df.collect()[0][f"{col_name}_count"]  # Contagem total (não nulos)
    col_distinct = metadata_df.collect()[0][f"{col_name}_distinct"]  # Valores distintos
    col_nulls = metadata_df.collect()[0][f"{col_name}_nulls"]  # Contagem correta de valores nulos
    col_min = metadata_df.collect()[0][f"{col_name}_min"] if f"{col_name}_min" in metadata_df.columns else "N/A"
    col_max = metadata_df.collect()[0][f"{col_name}_max"] if f"{col_name}_max" in metadata_df.columns else "N/A"
    col_duplicates = metadata_df.collect()[0][f"{col_name}_duplicates"] if f"{col_name}_duplicates" in metadata_df.columns else 0  # Contagem de duplicados

    # Buscar definição, regra de transformação e linhagem dos dados no dicionário
    col_definition, transformations_applied, lineage_info = col_descriptions.get(
        col_name, (f"Descrição da coluna {col_name}", "Nenhuma alteração aplicada.", "Origem desconhecida")
    )

    # Definir observações sobre a coluna (pode ser preenchido manualmente se necessário)
    comments = observacoes.get(col_name, "Sem observações adicionais.")

    # Adicionar à lista
    catalog_data.append(Row(ID_Coluna=index,
                            Coluna=col_name, 
                            Tipo_de_Dado=col_type, 
                            Definição=col_definition,
                            Valores_Min_Max=f"{col_min} | {col_max}", 
                            Qtd_Valores_Registrados=col_count,
                            Qtd_Valores_Nulos=col_nulls,
                            Qtd_Valores_Distintos=col_distinct,
                            Qtd_Valores_Duplicados=col_duplicates,
                            Regras_de_Transformação=transformations_applied,
                            Linhagem_Dados=lineage_info,  # ✅ Agora vem direto do dicionário!
                            Observacoes=comments,
                            Fonte="Kaggle",
                            Link_Fonte="https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship",
                            Arquivo_Original="BRA.csv",
                            Arquivo_S3="BrasilSerieA_2024_TodasPartidas.csv"
                            ))                            

# Criar o DataFrame do Catálogo de Dados
catalog_df = spark.createDataFrame(catalog_data)


#### 🔹 Salvando o Catálogo no S3 camada _Bronze_

In [0]:
# Caminho no S3 para salvar o Catálogo Estatístico
delta_catalogo_estatistico_path = "s3://mvp-brasileirao-2024/silver/delta/catalog_todas_partidas"

dbutils.fs.rm(delta_catalogo_estatistico_path, recurse=True)
print("🚨 Arquivos antigos do Delta removidos com sucesso!")

# Salvar a tabela Delta com as estatísticas
catalog_df.write.format("delta").mode("overwrite").save(delta_catalogo_estatistico_path)

print("✅ Catálogo Estatístico salvo no formato Delta com sucesso!")

🚨 Arquivos antigos do Delta removidos com sucesso!
✅ Catálogo Estatístico salvo no formato Delta com sucesso!


#### 🔹 Criando Tabela de Catálogos no Hive

In [0]:
%sql
DROP TABLE IF EXISTS catalogo_de_dados.catalog_todas_partidas_silver;

CREATE TABLE catalogo_de_dados.catalog_todas_partidas_silver
USING DELTA
LOCATION 's3://mvp-brasileirao-2024/silver/delta/catalog_todas_partidas';

#### 🔹 Exibindo o Catálogo de Dados via SQL

In [0]:
%sql
select * from catalogo_de_dados.catalog_todas_partidas_silver

ID_Coluna,Coluna,Tipo_de_Dado,Definição,Valores_Min_Max,Qtd_Valores_Registrados,Qtd_Valores_Nulos,Qtd_Valores_Distintos,Qtd_Valores_Duplicados,Regras_de_Transformação,Linhagem_Dados,Observacoes,Fonte,Link_Fonte,Arquivo_Original,Arquivo_S3
4,Data,date,Data em que a partida foi disputada,2024-04-13 | 2024-12-08,760,0,110,650,"Renomeação da coluna e conversão para formato de data | Ajuste das datas das partidas não encontradas na base Estatistica por Jogador (-1 dia, quando necessário)",Bronze → Silver,"⚠️ Ajuste manual: Datas ajustadas para garantir a correspondência entre os arquivos de partidas e estatísticas. Quando necessário, foi subtraído 1 dia das partidas não encontradas na estatística.",Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,BrasilSerieA_2024_TodasPartidas.csv
9,MandanteVisitante,string,Indica se o time jogou em casa ('Mandante') ou fora ('Visitante'),Mandante | Visitante,760,0,2,758,Transformado a partir dos DataFrames 'df_home' e 'df_away' e criado a partir da união de ambos,Silver (Transformação),Sem observações adicionais.,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,BrasilSerieA_2024_TodasPartidas.csv
12,Match_ID,string,Identificador único da partida,Match_0 | Match_99,760,0,380,380,Criado concatenando 'Match_' com um ID incremental,Silver (Coluna nova),Sem observações adicionais.,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,BrasilSerieA_2024_TodasPartidas.csv
13,Status_Ajustes,string,Indica se a data da partida foi ajustada,Ajustado -1 dia | Sem modificação,760,0,2,758,Criado para identificar alterações na data das partidas. Valores possíveis: 'Ajustado -1 dia' ou 'Sem modificação',Silver (Coluna nova),Sem observações adicionais.,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,BrasilSerieA_2024_TodasPartidas.csv
7,Turno,string,Indicação se a partida foi no primeiro ou segundo turno do campeonato,Primeiro Turno | Segundo Turno,760,0,2,758,Criado com base na data do jogo,Silver (Coluna nova),Sem observações adicionais.,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,BrasilSerieA_2024_TodasPartidas.csv
8,Clube,string,Nome do time participante da partida (mandante ou visitante),Athletico-PR | Vasco da Gama,760,0,20,740,"Criado a partir das colunas Home e Away, e unificando em uma única coluna",Silver (Transformação),Sem observações adicionais.,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,BrasilSerieA_2024_TodasPartidas.csv
10,Goals,int,Número de gols marcados pelo time na partida,0 | 6,760,0,7,753,Criado unificando HG (gols do mandante) e AG (gols do visitante),Silver (Coluna nova),Sem observações adicionais.,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,BrasilSerieA_2024_TodasPartidas.csv
11,Resultado,string,"Resultado da partida do ponto de vista do time (Vitória, Derrota ou Empate)",Derrota | Vitoria,760,0,3,757,Ajustado conforme a posição do time na partida,Silver (Transformação),Sem observações adicionais.,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,BrasilSerieA_2024_TodasPartidas.csv
2,Liga,string,Nome da liga do campeonato,Serie A | Serie A,760,0,1,759,Renomeação da coluna para padronização,Bronze → Silver,Sem observações adicionais.,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,BrasilSerieA_2024_TodasPartidas.csv
3,Temporada,int,Ano da temporada do campeonato,2024 | 2024,760,0,1,759,Renomeação da coluna para padronização | Filtragem para manter apenas 2024,Bronze → Silver,Sem observações adicionais.,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,BrasilSerieA_2024_TodasPartidas.csv


## 🔷 brasil_seriea_2024_silver_estatistica_jogador

####🔹 Criando o Metadata do dataframe

In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import col, count, countDistinct, min, max, sum

# Nome da tabela que queremos analisar (mudar para cada dataset da Silver)
tabela_silver = "default.brasil_seriea_2024_silver_estatistica_jogador"

# Ler os dados da Silver diretamente do Metastore
df = spark.read.table(tabela_silver)

# Criar DataFrame com estatísticas (incluindo contagem de nulos correta e duplicatas)
metadata_df = df.agg(
    *[count(col(c)).alias(f"{c}_count") for c in df.columns],  # Contagem total de registros NÃO NULOS
    *[countDistinct(col(c)).alias(f"{c}_distinct") for c in df.columns],  # Contagem de valores distintos
    *[min(col(c)).alias(f"{c}_min") for c in df.columns],  # Valor mínimo
    *[max(col(c)).alias(f"{c}_max") for c in df.columns],  # Valor máximo
    *[sum(col(c).isNull().cast("int")).alias(f"{c}_nulls") for c in df.columns],  # Contagem correta de nulos
    *[(count(col(c)) - countDistinct(col(c))).alias(f"{c}_duplicates") for c in df.columns]  # Contagem de valores duplicados
)

#### 🔹 Criação do Dicionário de descrições

In [0]:
col_descriptions = {
    # Colunas originais da Bronze (renomeadas na Silver)
    "Jogador": ("Nome do jogador que participou da partida", "Nenhuma alteração aplicada", "Bronze → Silver"),
    "Clube": ("Nome do clube do jogador", "Renomeação da coluna para padronização", "Bronze → Silver"),
    "Numero_Camisa": ("Número da camisa do jogador na partida", "Renomeação da coluna para padronização", "Bronze → Silver"),
    "Nacionalidade": ("Nacionalidade do jogador", "Renomeação da coluna para padronização | Valores nulos preenchidos com 'N/D' para manter padrão de 3 caracteres", "Bronze → Silver"),
    "Posicao_em_Campo": ("Posição do jogador dentro de campo", "Renomeação da coluna para padronização", "Bronze → Silver"),
    "idade": ("Idade do jogador na data da partida (anos e dias)", "Extraídos apenas os dois primeiros caracteres para padronização", "Bronze → Silver"),
    
    # 🔹 Atualização da coluna "Minutos_em_Campo"
    "Minutos_em_Campo": ("Minutos jogados pelo atleta na partida", 
                         "Renomeação da coluna para padronização | Valores nulos preenchidos com 0 para representar jogadores que não entraram | Conversão do tipo de dado para double", 
                         "Bronze → Silver"),
    
    # Estatísticas de gols e assistências
    "Gols": ("Número de gols marcados na partida pelo jogador", "Nenhuma alteração aplicada", "Bronze → Silver"),
    "Assitencias": ("Número de assistências realizadas pelo jogador", "Renomeação da coluna para padronização", "Bronze → Silver"),

    # Estatísticas de pênaltis e chutes
    "Penaltis_Batidos": ("Pênaltis cobrados pelo jogador", "Renomeação da coluna para padronização", "Bronze → Silver"),
    "Penaltis_Tentados": ("Pênaltis tentados pelo jogador (considerando acertos e erros)", "Renomeação da coluna para padronização", "Bronze → Silver"),
    "Total_Chutes": ("Total de chutes realizados (não inclui cobranças de pênalti)", "Renomeação da coluna para padronização", "Bronze → Silver"),
    "Chutes_a_Gol": ("Chutes a gol (não inclui cobranças de pênalti)", "Renomeação da coluna para padronização", "Bronze → Silver"),

    # Estatísticas disciplinares
    "Cartao_Amarelo": ("Quantidade de cartões amarelos recebidos", "Renomeação da coluna para padronização", "Bronze → Silver"),
    "Cartao_Vermelho": ("Quantidade de cartões vermelhos recebidos", "Renomeação da coluna para padronização", "Bronze → Silver"),

    # Estatísticas defensivas
    "Toques_na_Bola": ("Número total de vezes que o jogador tocou na bola (conta-se receber, driblar e passar como um toque)", 
                       "Renomeação da coluna para padronização | Conversão do tipo de dado para double", 
                       "Bronze → Silver"),
    "Desarmes": ("Número de desarmes bem-sucedidos realizados pelo jogador", 
                 "Renomeação da coluna para padronização | Conversão do tipo de dado para double", 
                 "Bronze → Silver"),
    "Interceptacoes": ("Número de interceptações defensivas feitas pelo jogador", "Renomeação da coluna para padronização", "Bronze → Silver"),
    "Bloqueios": ("Número de bloqueios defensivos (interceptação de chutes, passes ou cruzamentos perigosos)", 
                  "Nenhuma alteração aplicada | Conversão do tipo de dado para double", 
                  "Bronze → Silver"),
    
    # Estatísticas de Expectativa
    "Gols_Esperados": ("Métrica que estima a probabilidade de um chute resultar em gol, com base em variáveis como distância do gol, ângulo, tipo de finalização, entre outros.",
        "Renomeação da coluna para padronização",
        "Bronze → Silver"),
    "Assistencias_Esperadas": ("Métrica que estima a probabilidade de um passe resultar em assistência, com base na qualidade da chance criada.",
        "Renomeação da coluna para padronização",
        "Bronze → Silver"),

    # Estatísticas de criação
    "Acoes_de_Criacao": ("Ações que resultaram em chutes a gol (Shot-Creating Actions)", 
                         "Renomeação da coluna para padronização | Conversão do tipo de dado para integer", 
                         "Bronze → Silver"),
    "Acoes_de_Gol": ("Ações que resultaram em gols (Goal-Creating Actions)", 
                     "Renomeação da coluna para padronização | Conversão do tipo de dado para double", 
                     "Bronze → Silver"),

    # Estatísticas de passes
    "Passes_Completos": ("Número de passes completados com sucesso", 
                         "Renomeação da coluna para padronização | Conversão do tipo de dado para double", 
                         "Bronze → Silver"),
    "Passes_Tentados": ("Total de passes tentados", 
                        "Renomeação da coluna para padronização | Conversão do tipo de dado para double", 
                        "Bronze → Silver"),
    "Prct_Passes_Certos": ("Percentual de passes completados com sucesso", 
                           "Renomeação da coluna para padronização | Removidos caracteres não numéricos e convertida para string", 
                           "Bronze → Silver"),
    "Passes_Progressivos": ("Passes progressivos (passes em direção ao gol adversário)", 
                            "Renomeação da coluna para padronização | Conversão do tipo de dado para double", 
                            "Bronze → Silver"),

    # Estatísticas de condução e drible
    "Conducoes_de_Bola": ("Número total de conduções de bola bem-sucedidas (domínio de bola com os pés)", 
                          "Renomeação da coluna para padronização | Conversão do tipo de dado para double", 
                          "Bronze → Silver"),
    "Conducoes_Progressivas": ("Conduções progressivas (conduções para frente que resultam em ganho de espaço)", 
                               "Renomeação da coluna para padronização | Conversão do tipo de dado para double", 
                               "Bronze → Silver"),
    "Dribles_Tentados": ("Número total de tentativas de drible", 
                         "Renomeação da coluna para padronização | Conversão do tipo de dado para double", 
                         "Bronze → Silver"),
    "Dribles_Completos": ("Número de dribles bem-sucedidos", 
                          "Renomeação da coluna para padronização | Conversão do tipo de dado para double", 
                          "Bronze → Silver"),

    # 🔹 Atualização da coluna "Data"
    "Data": ("Data da partida disputada", 
             "Conversão para formato de data | Ajuste dos registros ausentes adicionados manualmente", 
             "Bronze → Silver"),

    # 🔹 Nova coluna "Origem_Dado"
    "Origem_Dado": ("Criada para identificar se os registros são originais da base ou foram adicionados manualmente. Valores possíveis: 'Original' ou 'Input Manual", 
                    "Nenhuma alteração aplicada, além da criação '", 
                    "Silver (Coluna nova)")
}


In [0]:
observacoes = {
    "idade": "⚠️ Ajuste manual: Preenchidos valores faltantes para o jogador Felipe Vieira em partidas específicas. Fontes Externas confirmadas: https://www.ogol.com.br/jogador/felipe-andrade/620537?epoca_id=153",
    "Data": "⚠️ Ajuste manual: Datas ajustadas e registros ausentes incluídos manualmente para garantir consistência com os dados de partidas."
}


#### 🔹 Criando o Dataframe do Catálogo

In [0]:
# Criar lista de metadados para o Catálogo de Dados
catalog_data = []

# Iterar sobre as colunas da tabela Silver
for index, col_name in enumerate(df.columns, start=1):
    col_type = df.select(col(col_name)).schema.fields[0].dataType.simpleString()  # Tipo da coluna
    col_count = metadata_df.collect()[0][f"{col_name}_count"]  # Contagem total (não nulos)
    col_distinct = metadata_df.collect()[0][f"{col_name}_distinct"]  # Valores distintos
    col_nulls = metadata_df.collect()[0][f"{col_name}_nulls"]  # Contagem correta de valores nulos
    col_min = metadata_df.collect()[0][f"{col_name}_min"] if f"{col_name}_min" in metadata_df.columns else "N/A"
    col_max = metadata_df.collect()[0][f"{col_name}_max"] if f"{col_name}_max" in metadata_df.columns else "N/A"
    col_duplicates = metadata_df.collect()[0][f"{col_name}_duplicates"] if f"{col_name}_duplicates" in metadata_df.columns else 0  # Contagem de duplicados

    # Buscar definição, regra de transformação e linhagem dos dados no dicionário
    col_definition, transformations_applied, lineage_info = col_descriptions.get(
        col_name, (f"Descrição da coluna {col_name}", "Nenhuma alteração aplicada.", "Origem desconhecida")
    )

    # Definir observações sobre a coluna (pode ser preenchido manualmente se necessário)
    comments = observacoes.get(col_name, "⚠️ Ajuste manual: Registros adicionados manualmente a partir do arquivo 'BrasilSerieA_2024_EstatisticaJogador_InputManual.csv' para garantir consistência dos dados")

    # Adicionar à lista
    catalog_data.append(Row(ID_Coluna=index,
                            Coluna=col_name, 
                            Tipo_de_Dado=col_type, 
                            Definição=col_definition,
                            Valores_Min_Max=f"{col_min} | {col_max}", 
                            Qtd_Valores_Registrados=col_count,
                            Qtd_Valores_Nulos=col_nulls,
                            Qtd_Valores_Distintos=col_distinct,
                            Qtd_Valores_Duplicados=col_duplicates,
                            Regras_de_Transformação=transformations_applied,
                            Linhagem_Dados=lineage_info,  # ✅ Agora vem direto do dicionário!
                            Observacoes=comments,
                            Fonte="Kaggle",
                            Link_Fonte="https://www.kaggle.com/datasets/eduardopalmieri/brasileiro-player-stats-2024",
                            Arquivo_Original= "database.csv",
                            Arquivo_S3 = "BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv"
                        ))                        

# Criar o DataFrame do Catálogo de Dados
catalog_df = spark.createDataFrame(catalog_data)


#### 🔹 Salvando o Catálogo no S3 camada _Bronze_

In [0]:
# Caminho no S3 para salvar o Catálogo Estatístico
delta_catalogo_estatistico_path = "s3://mvp-brasileirao-2024/silver/delta/catalog_estatistica_jogador"

dbutils.fs.rm(delta_catalogo_estatistico_path, recurse=True)
print("🚨 Arquivos antigos do Delta removidos com sucesso!")

# Salvar a tabela Delta com as estatísticas
catalog_df.write.format("delta").mode("overwrite").save(delta_catalogo_estatistico_path)

print("✅ Catálogo Estatístico salvo no formato Delta com sucesso!")

🚨 Arquivos antigos do Delta removidos com sucesso!
✅ Catálogo Estatístico salvo no formato Delta com sucesso!


#### 🔹 Criando Tabela de Catálogos no Hive

In [0]:
%sql
DROP TABLE IF EXISTS catalogo_de_dados.catalog_estatistica_jogador_silver;

CREATE TABLE catalogo_de_dados.catalog_estatistica_jogador_silver
USING DELTA
LOCATION 's3://mvp-brasileirao-2024/silver/delta/catalog_estatistica_jogador';

#### 🔹 Exibindo o Catálogo de Dados via SQL

In [0]:
%sql
select * from catalogo_de_dados.catalog_estatistica_jogador_silver

ID_Coluna,Coluna,Tipo_de_Dado,Definição,Valores_Min_Max,Qtd_Valores_Registrados,Qtd_Valores_Nulos,Qtd_Valores_Distintos,Qtd_Valores_Duplicados,Regras_de_Transformação,Linhagem_Dados,Observacoes,Fonte,Link_Fonte,Arquivo_Original,Arquivo_S3
17,Desarmes,double,Número de desarmes bem-sucedidos realizados pelo jogador,0.0 | 12.0,11746,0,13,11733,Renomeação da coluna para padronização | Conversão do tipo de dado para double,Bronze → Silver,⚠️ Ajuste manual: Registros adicionados manualmente a partir do arquivo 'BrasilSerieA_2024_EstatisticaJogador_InputManual.csv' para garantir consistência dos dados,Kaggle,https://www.kaggle.com/datasets/eduardopalmieri/brasileiro-player-stats-2024,database.csv,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv
18,Interceptacoes,int,Número de interceptações defensivas feitas pelo jogador,0 | 8,11746,0,9,11737,Renomeação da coluna para padronização,Bronze → Silver,⚠️ Ajuste manual: Registros adicionados manualmente a partir do arquivo 'BrasilSerieA_2024_EstatisticaJogador_InputManual.csv' para garantir consistência dos dados,Kaggle,https://www.kaggle.com/datasets/eduardopalmieri/brasileiro-player-stats-2024,database.csv,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv
19,Bloqueios,double,"Número de bloqueios defensivos (interceptação de chutes, passes ou cruzamentos perigosos)",0.0 | 7.0,11746,0,8,11738,Nenhuma alteração aplicada | Conversão do tipo de dado para double,Bronze → Silver,⚠️ Ajuste manual: Registros adicionados manualmente a partir do arquivo 'BrasilSerieA_2024_EstatisticaJogador_InputManual.csv' para garantir consistência dos dados,Kaggle,https://www.kaggle.com/datasets/eduardopalmieri/brasileiro-player-stats-2024,database.csv,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv
20,Gols_Esperados,double,"Métrica que estima a probabilidade de um chute resultar em gol, com base em variáveis como distância do gol, ângulo, tipo de finalização, entre outros.",0.0 | 1.7,11731,15,18,11713,Renomeação da coluna para padronização,Bronze → Silver,⚠️ Ajuste manual: Registros adicionados manualmente a partir do arquivo 'BrasilSerieA_2024_EstatisticaJogador_InputManual.csv' para garantir consistência dos dados,Kaggle,https://www.kaggle.com/datasets/eduardopalmieri/brasileiro-player-stats-2024,database.csv,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv
25,Passes_Tentados,double,Total de passes tentados,0.0 | 130.0,11746,0,118,11628,Renomeação da coluna para padronização | Conversão do tipo de dado para double,Bronze → Silver,⚠️ Ajuste manual: Registros adicionados manualmente a partir do arquivo 'BrasilSerieA_2024_EstatisticaJogador_InputManual.csv' para garantir consistência dos dados,Kaggle,https://www.kaggle.com/datasets/eduardopalmieri/brasileiro-player-stats-2024,database.csv,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv
26,Prct_Passes_Certos,double,Percentual de passes completados com sucesso,0.0 | 991.0,11696,50,466,11230,Renomeação da coluna para padronização | Removidos caracteres não numéricos e convertida para string,Bronze → Silver,⚠️ Ajuste manual: Registros adicionados manualmente a partir do arquivo 'BrasilSerieA_2024_EstatisticaJogador_InputManual.csv' para garantir consistência dos dados,Kaggle,https://www.kaggle.com/datasets/eduardopalmieri/brasileiro-player-stats-2024,database.csv,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv
27,Passes_Progressivos,double,Passes progressivos (passes em direção ao gol adversário),0.0 | 27.0,11746,0,24,11722,Renomeação da coluna para padronização | Conversão do tipo de dado para double,Bronze → Silver,⚠️ Ajuste manual: Registros adicionados manualmente a partir do arquivo 'BrasilSerieA_2024_EstatisticaJogador_InputManual.csv' para garantir consistência dos dados,Kaggle,https://www.kaggle.com/datasets/eduardopalmieri/brasileiro-player-stats-2024,database.csv,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv
28,Conducoes_de_Bola,double,Número total de conduções de bola bem-sucedidas (domínio de bola com os pés),0.0 | 106.0,11746,0,94,116